In [184]:
import re


def parse(text):
    sections = text.split("\n\n")
    seeds, maps = [int(n) for n in re.findall(r"\d+", sections[0])], []
    for section in sections[1:]:
        maps.append([])
        for l in section.split("\n")[1:]:
            maps[-1].append(tuple(int(i) for i in l.split()))
    return seeds, maps

In [185]:
input = open("05.txt").read()

seeds, maps = parse(input)

In [186]:
import re


def shift_range(_range: tuple, maps: list[tuple]) -> int | tuple:
    if not maps:
        return _range[0]

    for destination, start, span in maps[0]:
        # range in map
        if start <= _range[0] < start + span and start <= _range[1] < start + span:
            return shift_range(
                (destination + _range[0] - start, destination + _range[1] - start),
                maps[1:],
            )

        # range starts in map but ends outside
        if start <= _range[0] < start + span and start + span < _range[1]:
            return min(
                shift_range(
                    (destination + _range[0] - start, destination + span), maps[1:]
                ),
                shift_range((start + span, _range[1]), maps),
            )

        # range starts in outside but ends in map
        if _range[0] < start and start <= _range[1] < start + span:
            return min(
                shift_range((_range[0], start - 1), maps),
                shift_range((destination, destination + _range[1] - start), maps[1:]),
            )

        # range outside map with map in between
        if _range[0] < start and start + span < _range[1]:
            return min(
                shift_range((_range[0], start - 1), maps),
                shift_range((destination, destination + span), maps[1:]),
                shift_range((start + span, _range[1]), maps),
            )

    # range outside map with no overlap
    return shift_range(_range, maps[1:])

In [187]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

test_seeds, test_maps = parse(test_input)

seed_ranges = [(i, i) for i in test_seeds]

assert min((shift_range(_range, test_maps) for _range in seed_ranges)) == 35

In [188]:
seed_ranges = [(i, i) for i in seeds]

print(min((shift_range(_range, maps) for _range in seed_ranges)))

214922730


In [189]:
seed_ranges = [(seeds[i], seeds[i] + seeds[i+1] - 1) for i in range(0, len(seeds), 2)]

print(min((shift_range(_range, maps) for _range in seed_ranges)))

148041808
